In [ ]:
import mlflow.keras
import mlflow
import mlflow.tensorflow
from sklearn.model_selection import train_test_split, cross_val_score

from table_functions import *
from analyze_visualisation import *
from ml_models.decision_tree import *
from ml_models.random_forest import *
from ml_models.xg_boost import *
from ml_models.neural_network_train import *
from ml_models.extra_random_forest import *
from ml_functions import *

In [ ]:
# TO MUSI BYC USTAWIONE ZAWSZE TAK SAMO, a potem jak sie chce umieszczac dane modele w innym folderze to zmienia sie nazwe  artifact_directory = "other_classification"
# jak chce sie wlaczyc potem mlflow w przegladarce, trzeba w terminalu przejsc do lokalizacji 'src' i wpisac komende mlflow ui
current_working_directory = os.getcwd()
mlflow.set_tracking_uri(f"file://{os.path.join(current_working_directory, 'mlruns')}")

In [ ]:
data_keys = ['x_train', 'x_valid', 'x_test', 'y_train', 'y_valid', 'y_test']
ml_data = {key: None for key in data_keys}
for key in ml_data:
    ml_data[key] = read_csv(f'{key}.csv')

In [ ]:
ml_data_no_cat = ml_data.copy()
categorical_columns_ = list(ml_data_no_cat['x_train'].iloc[:, 130:].columns)
for cold in ['x_train', 'x_valid', 'x_test']:
    ml_data_no_cat[cold].drop(columns=categorical_columns_, inplace=True)

In [ ]:
ml_data_dgm = ml_data.copy()
for cold in ['x_train', 'x_valid', 'x_test']:
    ml_data_dgm[cold] = ml_data_dgm[cold].loc[:, :'czas_taktu']

In [ ]:
final_table = read_csv('final_table_before_normalization.csv')
ml_data_small = split_data(final_table, nok_samples=100000, ok_samples=100000)
ml_data_small['x_train'], scaler = normalize_data(ml_data_small['x_train'])
ml_data_small['x_valid'] = normalize_data(ml_data_small['x_valid'], scaler)
ml_data_small['x_test'] = normalize_data(ml_data_small['x_test'], scaler)

In [ ]:
pd.set_option('display.max_rows', None)  # Set to None for unlimited rows
pd.set_option('display.max_columns', None)  # Set to None for unlimited columns
final_table.head(3)

In [ ]:
final_table = read_csv('final_table_before_normalization.csv')
few_columns_df = final_table[['cisnienie_tloka', 'cisnienie_koncowe', 'nachdruck_hub', 'anguss', 'our_final_status']]
ml_data_few_col = split_data(few_columns_df, nok_samples=200000, ok_samples=200000)
ml_data_few_col_norm = ml_data_few_col.copy()
ml_data_few_col_norm['x_train'], scaler = normalize_data(ml_data_few_col_norm['x_train'])
ml_data_few_col_norm['x_valid'] = normalize_data(ml_data_few_col_norm['x_valid'], scaler)
ml_data_few_col_norm['x_test'] = normalize_data(ml_data_few_col_norm['x_test'], scaler)

<h4>Drzewo decyzyjne</h4>

In [ ]:
clf = decision_tree_model(*ml_data.values(), run_name_="normal_run")
#print_stats(clf, ml_data['x_test'], ml_data['y_test'])

<h4>Las losowy<h4>

In [ ]:
clf = random_forest_model(*ml_data_few_col.values(), run_name_="only_4_columns_1")
clf = random_forest_model(*ml_data_few_col_norm.values(), run_name_="only_4_columns_1_norm")

<h4>XGBoost</h4>

In [ ]:
clf = xgb_model(*ml_data_small.values(), run_name_="ml_data_small")
#clf = xgb_model(*ml_data_few_col_norm.values(), run_name_="only_4_columns_1_norm")

<h4>Sieci neuronowe<h4>

In [ ]:
for umap_c in [None, 10, 20, 100, 150]:
    if umap_c is not None:
        for umap_dist in [0.75, 0.5, 0.25, 0.1, 0.01]:
            ml_data_c = ml_data.copy()
            ml_data_c['x_train'], ml_data_c['x_valid'], ml_data_c['x_test'] = umap_transformation(ml_data_c['x_train'], ml_data_c['x_valid'], ml_data_c['x_test'],
                                                                                                   n_components_umap= umap_c, umap_min_dist= umap_dist)
            for batch_ in [16, 32, 64, 128, 256, 512]:
                for model_num in [1, 3, 4, 5, 6]:
                    if model_num == 4:
                        for drop_ in [0.5, 0.25]:
                            compile_fit_evaluate_model(*ml_data_c.values(), epochs_=100, batch_size_= batch_, model_number=model_num, drop_neurons=drop_,
                                                        run_name_=f"aa_{umap_c}u_{umap_dist}dst_{batch_}b_{model_num}m_{drop_}d")
                    else:
                        compile_fit_evaluate_model(*ml_data_c.values(), epochs_=100, batch_size_= batch_, model_number=model_num,
                            run_name_=f"aa_{umap_c}u_{umap_dist}dst_{batch_}b_{model_num}m")
    else:
        for batch_ in [16, 32, 64, 128, 256, 512]:
            for model_num in [1, 3, 4, 5, 6]:
                if model_num == 4:
                    for drop_ in [0.5, 0.25]:
                        compile_fit_evaluate_model(*ml_data.values(), epochs_=100, batch_size_= batch_, model_number=model_num, drop_neurons=drop_,
                            run_name_=f"aa_{batch_}b_{model_num}m_{drop_}d")
                else:
                    compile_fit_evaluate_model(*ml_data.values(), epochs_=100, batch_size_= batch_, model_number=model_num,
                            run_name_=f"aa_{batch_}b_{model_num}m")


<h4>Super drzewa losowe</h4>

In [ ]:
clf = extra_random_trees(*ml_data.values(), run_name_="first_run")

<h4>Cross Validacja</h4>

Według mnie na razie zostawmy cross validacje w spokoju, na normalnym zbiorze znajdzmy najlepsze parametry dla modeli a potem dopiero zrobmy na takich parametrach cross validacje by zobaczyc 'prawdziwa' jakosc modelu
PS. ponizszy kod wykonuje sie jakas 1h
PS2. dane sie zapisaly ale jakis syf

In [ ]:
artifact_directory="cv_random_forest"
mlflow.set_experiment(artifact_directory)
mlflow.sklearn.autolog()

with start_run(run_name="testowy run"):
    clf = RandomForestClassifier(random_state=0)
    scores = cross_val_score(estimator=clf, X=ml_data['x_train'], y=ml_data['y_train'], cv=5)
print(scores)
# [0.99786842 0.99779825 0.99742105 0.99770175 0.99797368] - wynik

In [ ]:
loaded_model = mlflow.keras.load_model(r'file://C:\Users\dlxpmx8\Desktop\Projekt_AI\meb_process_data_analysis\src\mlruns/386163800829212723/5e018f9f70514ad889d242edd0430b67/artifacts/model')

In [ ]:
distribution_of_probability(ml_data['x_test'], ml_data['y_test'], model_=loaded_model)